# Notebook 1 : Dataset Preprocessing

Began 3 March 2020 for 2 weeks by Amaury de Barbuat from ECL

Updated by William Riou from ENSTA PARIS

## 0. Intro

The goal of this Notebook is to do multiple modifications of the initial dataset, using pandas library to manipulate Dataframes, in order to use the file on SumUp Nucleus. To do so, the final goal is to get a file with the following columns : ['Title','Content','Time'] where title refers to synopsis in TEXT.csv, content refers to narrative in TEXT.csv and time refers to date in ALL_ITEMS.csv.

This process is very spacedisk and time consumer. It is not optimized.

In [2]:
# Classic useful libraries

import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import math
import time

## 1. Pre-processing of TEXT.csv

File requires several modifications to be usable for step 3.

### 11- N1/2/3/4/5 and Callbacks

In [4]:
# First mods detailed as following :
# a. Rows with 2 cells turned into 1. The writer used a ";" in is report wich creates a confusion for the csv format.
# To overpass it we replace ";" by "." in essays so that we do not lose information and make the file useable.
# b. Narrative -> N and Synopsis -> S
# c. When author gave multiple narrative, reunite N and N2
input_file="given_data_sets/TEXT.csv"
data0=pd.read_csv(input_file,sep='|',usecols=[0, 3, 4])
data0=data0.replace(';','.',regex=True)
data0=data0.replace('Narrative','N',regex=True)
data0=data0.replace('Synopsis','S',regex=True)
n0 = len(data0)
# print(n0)
for i in range(n0):
    if data0["ATTRIBUTE"][i] == "N2" :
        A = data0["TEXT"][i-1]
        B = data0["TEXT"][i]
        data0["TEXT"][i-1] = A+' '+B
        data0=data0.drop([i])

data0.to_csv('0_TEXT.csv',sep='|', header=True, index=False)

/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# When author gave multiple narrative, reunite N and N3

data1=pd.read_csv('0_TEXT.csv',sep='|')
n1 = len(data1)
# print(n1)
for i in range(n1):
    if data1["ATTRIBUTE"][i] == "N3" :
        A = data1["TEXT"][i-1]
        B = data1["TEXT"][i]
        data1["TEXT"][i-1] = A+' '+B
        data1=data1.drop([i])
data1.to_csv('1_TEXT.csv',sep='|', header=True, index=False)

/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
# When author gave multiple narrative, reunite N and N4

data2=pd.read_csv('1_TEXT.csv',sep='|')
n2=len(data2)
#print("Taille du dataset:",len(data2))
for i in range(n2):
    if data2["ATTRIBUTE"][i] == "N4" :
        A = data2["TEXT"][i-1]
        B = data2["TEXT"][i]
        data2["TEXT"][i-1] = A+' '+B
        data2=data2.drop([i])
data2.to_csv('2_TEXT.csv',sep='|', header=True, index=False)

Taille du dataset: 430222


/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
# When author gave multiple narrative, reunite N and N5

data3=pd.read_csv('2_TEXT.csv',sep='|')
#print("Taille du dataset:",len(data3))

for i in range(n3):
    if data3["ATTRIBUTE"][i] == "N5" :
        A = data3["TEXT"][i-1]
        B = data3["TEXT"][i]
        data3["TEXT"][i-1] = A+' '+B
        data3=data3.drop([i])
data3.to_csv('3_TEXT.csv',sep='|', header=True, index=False)

taille du dataset: 430198


/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
# Calculate the number of "Callback" rows in the file.
# We won't consider Callbacks for this study as there are so few and they don't add much information.

file=pd.read_csv('3_TEXT.csv',sep='|')
n=len(file)
S=0
for i in range (n):
    if file['ATTRIBUTE'][i] == 'Callback':
        S=S+1
#print(S)

2640


In [9]:
# Creation of the 4_TEXT.csv file where callback rows have been removed.

data4=pd.read_csv('3_TEXT.csv',sep='|')
n4=len(data4)
#print(n4)
for i in range(n4):
    if data4["ATTRIBUTE"][i] == "Callback" :
        data4=data4.drop([i])
data4.to_csv('4_TEXT.csv',sep='|', header=True, index=False)

430196


## 2- Multiple cleaning

In [10]:
# Cleaning of the dataset : Goal is to have the same pattern for the file, e.g. given an ITEM_ID that the two rows
# well correspond to NARRATIVE then SYNOPSIS (if only one row : id / NaN / Content)

data5=pd.read_csv('4_TEXT.csv',sep='|')
n5=len(data5)
#print(n5)
i=2
while i < n5-1:
    
    if data5["ATTRIBUTE"][i] == "N" and data5["ATTRIBUTE"][i+1] == "N" and data5["ITEM_ID"][i-2] == data5["ITEM_ID"][i] :
        A = data5["TEXT"][i-2]
        B = data5["TEXT"][i]
        data5["TEXT"][i-2] = A+' '+B
        data5 = data5.drop([i])
        i = i + 3
    
    elif data5["ATTRIBUTE"][i] == "N" and data5["ATTRIBUTE"][i+1] == "N" and data5["ITEM_ID"][i-2] != data5["ITEM_ID"][i] and data5["ITEM_ID"][i+1] != data5["ITEM_ID"][i] :
        data5["ATTRIBUTE"][i] = "nan"
        i = i + 1
    
    else :
        i = i + 1
        
#print(len(data5))
data5.to_csv('5_TEXT.csv',sep='|', header=True, index=False)

427556


/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


414572


In [13]:
# Because of the needed i+3 in the loop of the previous code, we have to re-use the same code to treat the cases 
# that have not been treated yet.
# note that we have ran this code multiple times to eliminate the rows skipped by the +3 iterations...

data6=pd.read_csv('5_TEXT.csv',sep='|')
n6=len(data6)
#print(n6)
i=2
while i < n6-1:
    
    if data6["ATTRIBUTE"][i] == "N" and data6["ATTRIBUTE"][i+1] == "N" and data6["ITEM_ID"][i-2] == data6["ITEM_ID"][i] :
        A = data6["TEXT"][i-2]
        B = data6["TEXT"][i]
        data6["TEXT"][i-2] = A+' '+B
        data6 = data6.drop([i])
        i = i + 3
    
    elif data6["ATTRIBUTE"][i] == "N" and data6["ATTRIBUTE"][i+1] == "N" and data6["ITEM_ID"][i-2] != data6["ITEM_ID"][i] and data6["ITEM_ID"][i+1] != data6["ITEM_ID"][i] :
        data6["ATTRIBUTE"][i] = "NaN"
        i = i + 1
    
    elif data6["ATTRIBUTE"][i] == "S" and data6["ATTRIBUTE"][i+1] == "S" and data6["ITEM_ID"][i+1] != data6["ITEM_ID"][i] :
        data6["ATTRIBUTE"][i+1] = "NaN"
        i = i + 2
    
    else :
        i = i + 1
        
# print(len(data6))
data6.to_csv('9_TEXT.csv',sep='|', header=True, index=False)

414572


/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


413114


##  3- Final tests

In [14]:
# Test the quality of the dataset : are there still rows that don't respect the pattern we are looking for
# before running THE_FILE (the cleaned one)


data=pd.read_csv('9_TEXT.csv',sep='|')
n=len(data)
print(n)

Zn=0
Zs=0

Ln=[]
Ls=[]

for i in range (n-1):
    
    if data["ATTRIBUTE"][i] == "N" and data["ATTRIBUTE"][i+1] == "N" :
        #print(data["ITEM_ID"][i])
        Zn = Zn + 1
        Ln.append(data["ITEM_ID"][i])
    
    if data["ATTRIBUTE"][i] == "S" and data["ATTRIBUTE"][i+1] == "S" :
        #print(data["ITEM_ID"][i+1])
        Zs = Zs + 1
        Ls.append(data["ITEM_ID"][i+1])

print("Zn =",Zn)
print("Zs =",Zs)
print(Ln[0:10])
print(Ls)

413114
Zn = 357
Zs = 0
[838119, 838119, 840650, 842437, 846922, 848049, 850427, 853196, 854946, 855381]
[]


In [15]:
# Observation of the null value

test=pd.read_csv('9_TEXT.csv',sep='|')

n=len(test)

Z=0
L=[]

for i in range (n-1):
    
    if test['ATTRIBUTE'][i] != 'N' and test['ATTRIBUTE'][i] != 'S' :
        Z=Z+1
        L.append(test['ITEM_ID'][i])

#print("Z =",Z)
#print(L)

Z = 20
[252110, 269574, 269701, 270963, 274331, 534517, 534869, 545509, 623021, 716570, 726891, 783998, 817578, 829157, 920827, 951293, 1090400, 1315660, 1388808, 1615449]


In [16]:
# Null value in the file ?

test=pd.read_csv('9_TEXT.csv',sep='|')
test.isnull().values.any()

True

In [17]:
test=pd.read_csv('9_TEXT.csv',sep='|')

print(test['ATTRIBUTE'][75058])

np.isnan(test['ATTRIBUTE'][75058])


nan


True

## 4- Ultimate TEXT file - Ending up the preprocess

In [28]:
# Final modification for the text file : create one with columns ITEM_ID / TITLE (Synopsis or NaN) / CONTENT (Narrative)
tic=time.time()

new = pd.DataFrame(columns=['ITEM_ID', 'TITLE', 'CONTENT'])
old = pd.read_csv('9_TEXT.csv',sep='|')

l=len(old)
#print(l)

i=0
L=[]
S=0

while i < l-1:
    
    if old['ATTRIBUTE'][i] == 'N' and old['ATTRIBUTE'][i+1] == 'S' and old['ITEM_ID'][i] == old['ITEM_ID'][i+1] :
        new.loc[i] = [old['ITEM_ID'][i], old['TEXT'][i+1], old['TEXT'][i]]
        i=i+2
    
    else :
        new.loc[i] = [old['ITEM_ID'][i], old['ATTRIBUTE'][i], old['TEXT'][i]]
        L.append(old['ITEM_ID'][i])
        S=S+1
        i=i+1

#print(S)
#print(L)
#print(len(new))

tac=time.time()-tic
print(tac)

new.to_csv('sext.csv',sep='|', header=True, index=False)


413114
380
[252110, 269574, 269701, 270963, 274331, 534517, 534869, 545509, 623021, 716570, 726891, 783998, 817578, 829157, 838119, 838119, 840650, 842437, 846922, 848049, 850427, 853196, 854946, 855381, 859429, 859429, 865732, 868538, 870366, 870366, 879171, 890376, 910182, 910849, 910849, 911310, 912857, 917076, 917659, 917659, 918756, 920333, 920618, 920827, 922139, 924718, 926476, 930569, 932002, 933124, 933958, 938674, 941729, 944285, 948643, 951293, 952074, 957242, 961155, 961683, 966767, 972667, 975253, 975830, 981257, 981950, 982261, 982928, 983988, 987483, 988453, 991470, 991657, 994790, 994790, 1002555, 1002555, 1007796, 1008744, 1010715, 1013238, 1017622, 1018500, 1020720, 1024170, 1031251, 1032786, 1039409, 1042466, 1054698, 1056310, 1057181, 1058914, 1061147, 1072442, 1074684, 1085598, 1090400, 1092921, 1093775, 1096756, 1098463, 1099381, 1100689, 1101201, 1102906, 1103276, 1107149, 1108138, 1113934, 1114191, 1115865, 1118622, 1118664, 1119097, 1121478, 1122731, 1124461, 1

In [29]:
# Final touch to finally build new_TEXT.csv

data=pd.read_csv('sext.csv',sep='|')

n=len(data)
i=100000

while i < n-1 :
    
    if data['ITEM_ID'][i] == 534869 :
        data=data.drop([i])
        i=i+50000
    
    if data['ITEM_ID'][i] == 1283708 :
        A = data["CONTENT"][i]
        B = data["CONTENT"][i+1]
        data["CONTENT"][i] = A+' '+B
        data=data.drop([i+1])
        i=i+2
    
    if data['ITEM_ID'][i] == 1283723 :
        data['TITLE'][i] = 'NaN'
        i=i+1
    
    if data['ITEM_ID'][i] == 1388808 :
        data['TITLE'][i] = 'NaN'
        data=data.drop([i+1])
        i=n
    
    else :
        i=i+1

data.to_csv('new_TEXT.csv',sep='|', header=True, index=False)

/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/urendil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
# print(len(pd.read_csv('new_TEXT.csv',sep='|')))

206744


# 2- Pre-processing of ALL_ITEMS.csv

1. TEXT : items come from number 75 085 to 1 690 771 and there are only 2 rows per items (more if many writers)
2. ALL_ITEMS : items come from number 75 085 to 1 690 771 and approximately 50 rows per items

Creation of a file untitled new_ALL_ITEMS.csv with the following columns ['ITEM_ID', 'DATE']

In [31]:
# Only keep the item_id and date information from the ALL_ITEMS file
input_file1="given_data_sets/ALL_ITEMS.csv"
data=pd.read_csv(input_file1,sep='|',usecols=[0, 3, 4])

n=len(data)
L=[['ITEM_ID', 'DATE']]

for i in range(n) :
    if data["ATTRIBUTE"][i] == "Date" :
        L.append([data["ITEM_ID"][i], data["VALUE"][i]])

import csv

with open('1_ALL_ITEMS.csv','w', newline='') as fp :
    a = csv.writer(fp, delimiter = '|')
    a.writerows(L)

In [32]:
# Make sure that both files (text and time) have the same size, so that we can unify them

old=pd.read_csv('1_ALL_ITEMS.csv',sep='|')

n=len(old)
# print(n)
i=0

for i in range(n-1):
    
    if old['ITEM_ID'][i] == 534869:
        old=old.drop([i])
    
    elif old['ITEM_ID'][i] == 953498:
        old=old.drop([i])

# print(len(old))
old.to_csv('new_ALL_ITEMS.csv',sep='|', header=True, index=False)

206389
206387


## 3- Unification of both previous files

Goal is to create a single DataFrame with the following structure :
['TITLE', 'CONTENT', 'TIME']

In [33]:
# Creat THE_FILE
# To do so, first add a new column to the file, put the right data in its row and delete the item_id column

text = pd.read_csv('new_TEXT.csv',sep='|', usecols=[1, 2]) # item_id / title(synopsis) / content(narrative)
item = pd.read_csv('new_ALL_ITEMS.csv',sep='|', usecols=[1]) # item_id / date

text['TIME'] = item['DATE']

text.to_csv('data.csv',sep='|', header=True, index=False)

## 4- A file useable within SumUp's Nucleus
Just turn the separators of the file from '|' to ','.

In [34]:
# In this file, we kept the commas of the texts.

df=pd.read_csv('data.csv',sep='|')
df.to_csv('ASRS0.csv',sep=',',header=True, index=False)

In [35]:
# In this file we removed the commas from the texts.

df=pd.read_csv('data.csv',sep='|')
df=df.replace(',',' ',regex=True)
df.to_csv('ASRS.csv',sep=',',header=True, index=False)

### 41- Right date format and column order for nucleus requirements

In [7]:
# treating the 4 cases of the dataset where date format is yyyy and not yyyymm
import pandas as pd
data=pd.read_csv('ASRS0.csv',sep=',')

n=len(data['TIME'])
for i in range(n):
    if data['TIME'][i] < 3000 :
        # print(data['TIME'][i])
        data['TIME'][i] = data['TIME'][i-1]
        # print(data['TIME'][i])

data.to_csv('ASRS0.csv',sep=',',header=True, index=False)

In [37]:
data=pd.read_csv('ASRS0.csv',sep=',')

# columns : from TITLE, CONTENT, TIME to time, content, title
A=data['TITLE']
#print('1')
B=data['TIME']
#print('2')
data['TITLE']=B
#print('3')
data['TIME']=A
#print('4')
data=data.rename(columns={"TITLE": "time", "CONTENT": "content", "TIME": "title"})
#print('5')

# dates : from yyyymm to yyyy-mm-01
d='01'
data['time'] = pd.to_datetime(data['time'], format='%Y%m').dt.strftime("%Y-%m-%d")
#print('6')


data.to_csv('ASRS1.csv',sep=',',header=True, index=False)

1
2
3
4
5
6
